# CHAOS Stellar Mass Density Profiles

* [NGC0628](http://legacysurvey.org/viewer-dev?ra=24.1741665&dec=15.7834583&zoom=12&layer=decals-dr7&lslga)
* [NGC5194](http://legacysurvey.org/viewer-dev?ra=202.469547&dec=47.195151&zoom=11&layer=mzls+bass-dr6&lslga)
* [NGC5457](http://legacysurvey.org/viewer-dev?ra=210.802368&dec=54.349023&zoom=10&layer=mzls+bass-dr6&lslga)
* [NGC3184](http://legacysurvey.org/viewer-dev?ra=154.570590&dec=41.4243426&zoom=12&layer=mzls+bass-dr6&lslga)

In [1]:
import os, pdb
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import fitsio
from astropy.table import Column

In [2]:
import LSLGA.io
import LSLGA.misc
import LSLGA.qa

In [3]:
import multiprocessing
nproc = multiprocessing.cpu_count() // 2

In [4]:
%matplotlib inline

#### Specify the (SDSS) bands, the pixel scale, and the top-level data directory.

In [5]:
pixscale = 0.396 # [arcsec/pix]
band = ('g', 'r', 'i')

In [6]:
datadir = os.path.join( LSLGA.io.LSLGA_dir(), 'science', 'chaos' )

#### Read the sample.
Add some additional data we'll need to the output table.

In [32]:
def read_chaos(band=('g', 'r', 'i'), psfsize=1.3):
    """PSF size in FWHM arcsec is assumed to be constant.
    
    """
    sample = LSLGA.io.read_parent()
    these = np.hstack( [np.where(np.isin(sample['GALAXY'].data, chaosgal.encode('utf-8')))[0]
                        for chaosgal in ('NGC0628', 'NGC5194', 'NGC5457', 'NGC3184')] )
    sample = sample[these]

    # Add distances, the data release, the PSF widths, and the cutout size.
    sample.add_column(Column(name='DISTANCE', length=len(sample), dtype='f4'))
    dist = {'NGC0628': 7.2, 'NGC5194': 7.9, 'NGC5457': 7.4, 'NGC3184': 11.6} # [Mpc]    
    for ss in sample:
        ss['DISTANCE'] = dist[ss['GALAXY']] # [Mpc]
        
    sample.add_column(Column(name='DR', dtype='S3', length=len(sample)))
    gal2dr = {'NGC0628': 'DR7', 'NGC5194': 'DR6', 'NGC5457': 'DR6', 'NGC3184': 'DR6'}
    for ss in sample:
        ss['DR'] = gal2dr[ss['GALAXY']]
        
    # Cutout size (SDSS pixels).
    sample.add_column(Column(name='CUTOUT_SIZE', length=len(sample), dtype='i2'))
    gal2size = {'NGC0628': 2000, 'NGC5194': 2000, 'NGC5457': 3000, 'NGC3184': 1500}
    for ss in sample:
        ss['CUTOUT_SIZE'] = gal2size[ss['GALAXY']] # [SDSS pixels]
        
    # Add PSF width (nominal) and Galactic extinction values.
    for filt in band:
        sample['PSFSIZE_{}'.format(filt.upper())] = np.repeat(psfsize, len(sample))
    for filt in band:
        sample['MWDUST_A{}'.format(filt.upper())] = np.repeat(0.0, len(sample)) # initialize
    
    gal2dust = {'NGC0628': (0.232, 0.16, 0.119), # gri
                'NGC5194': (0.116, 0.08, 0.06), 
                'NGC5457': (0.028, 0.02, 0.015),
                'NGC3184': (0.055, 0.038, 0.028)}
    for ss in sample:
        for ii, filt in enumerate(band):
            ss['MWDUST_A{}'.format(filt.upper())] = gal2dust[ss['GALAXY']][ii]
        
    # Fix the radius and position angle of NGC5194
    #sample[]
        
    return sample

In [33]:
sample = read_chaos(band=band)
sample

LSLGA_ID,GALAXY,PGC,RA,DEC,TYPE,BAR,RING,MULTIPLE,COMPACTNESS,T,PA,D25,BA,DIAM_REF,Z,SB_D25,MAG,MAG_REF,WISE_RA,WISE_DEC,CNTR,W1MPRO,W1SIGMPRO,W2MPRO,W2SIGMPRO,W3MPRO,W3SIGMPRO,W4MPRO,W4SIGMPRO,RCHI2,CC_FLAGS,EXT_FLG,PH_QUAL,XSCPROX,W1RSEMI,W1BA,W1PA,W1GMAG,W1GERR,W2GMAG,W2GERR,W3GMAG,W3GERR,W4GMAG,W4GERR,IN_ALLWISE,IN_DESI,NEAR_BRIGHTSTAR,DISTANCE,DR,CUTOUT_SIZE,PSFSIZE_G,PSFSIZE_R,PSFSIZE_I,MWDUST_AG,MWDUST_AR,MWDUST_AI
int64,bytes29,int64,float64,float64,bytes4,bytes3,bytes3,bytes3,bytes3,float32,float32,float32,float32,bytes3,float32,float32,float32,bytes1,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float32,bytes4,int32,bytes4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,bool,bool,float32,bytes3,int16,float64,float64,float64,float64,float64,float64
518063,NGC0628,5974,24.174166500000002,15.783458300000001,Sc,nan,nan,nan,nan,5.2,nan,9.88553,0.94406086,iso,0.002196186,23.308481,9.705,B,24.1741387,15.7836868,234115101351047933,10.655,0.023,10.695,0.023,9.342,0.074,8.271,0.374,32.27,0000,5,AAAC,0.87,126.96,0.95,70.0,7.093,0.01,6.986,0.011,3.632,0.01,1.76,0.015,True,True,False,7.2,DR7,2000,1.3,1.3,1.3,0.232,0.16,0.119
1073844,NGC5194,47404,202.469547,47.195151,SABb,B,nan,M,nan,4.0,163.0,13.708821,0.85113806,iso,0.0015370633,22.921482,8.608,B,202.4696996,47.1951717,2029146901351042304,8.84,0.024,8.664,0.02,6.19,0.023,3.386,0.036,46.18,hhdd,5,AAAA,0.43,180.0,0.91,45.0,5.43,0.014,5.404,0.006,1.509,0.006,-0.28,0.03,True,True,False,7.9,DR6,2000,1.3,1.3,1.3,0.116,0.08,0.06
1237406,NGC5457,50063,210.80236799999997,54.349022999999995,SABc,B,nan,M,nan,5.9,nan,23.988337,0.9616123,iso,0.0007902133,23.88948,8.361,B,210.8021726,54.3487903,2119154501351013505,10.348,0.023,10.213,0.021,6.624,0.018,4.453,0.029,33.86,0000,5,AAAA,0.56,180.0,0.98,28.0,6.751,0.019,6.704,0.019,3.397,0.024,1.778,0.104,True,True,False,7.4,DR6,3000,1.3,1.3,1.3,0.028,0.02,0.015
1253132,NGC3184,30087,154.57058999999998,41.4243426,SABc,B,nan,nan,nan,5.9,nan,7.396052,0.97050995,iso,0.0019753666,23.384481,10.411,B,154.5705508,41.424279,1542140801351054772,10.97,0.023,10.88,0.021,7.418,0.021,4.711,0.03,23.9,0000,5,AAAA,0.69,115.66,0.97,60.0,7.652,0.011,7.517,0.011,4.133,0.008,2.339,0.017,True,True,False,11.6,DR6,1500,1.3,1.3,1.3,0.055,0.038,0.028


#### Download the SDSS imaging (if necessary).

In [9]:
def download_data(sample, pixscale=0.396, clobber=False):
    """Note that the cutout server has a maximum cutout size of 3000 pixels.
    
    montage -bordercolor white -borderwidth 1 -tile 2x2 -geometry +0+0 -resize 1024 \
      NGC0628-SDSS.jpg NGC3184-SDSS.jpg NGC5194-SDSS.jpg NGC5457-SDSS.jpg chaos.png
    """
    import subprocess
    
    for ss in sample:
        jpgfile = os.path.join(datadir, '{}-SDSS.jpg'.format(ss['GALAXY']))
        fitsfile = jpgfile.replace('.jpg', '.fits')
        if os.path.exists(jpgfile) and clobber is False:
            print('Done: {}'.format(jpgfile))
        else:
            cmd = 'wget -c -O {outfile}'
            cmd += '"http://legacysurvey.org/viewer/jpeg-cutout?ra={ra}dec={dec}&pixscale={pixscale}&size={size}&layer=sdss"'
            cmd = cmd.format(outfile=jpgfile, ra=ss['RA'], dec=ss['DEC'],
                             pixscale=pixscale, size=ss['CUTOUT_SIZE'])
            print(cmd)
            err = subprocess.call(cmd.split())
            
        if os.path.isfile(fitsfile) or not clobber:
            print('Done: {}'.format(fitsfile))
        else:
            cmd = cmd.replace('jpeg-cutout', 'fits-cutout')
            print(cmd)
            err = subprocess.call(cmd.split())

In [10]:
download_data(sample, pixscale=pixscale)

Done: /Users/ioannis/research/projects/LSLGA/science/chaos/NGC0628-SDSS.jpg
Done: /Users/ioannis/research/projects/LSLGA/science/chaos/NGC0628-SDSS.fits
Done: /Users/ioannis/research/projects/LSLGA/science/chaos/NGC5194-SDSS.jpg
Done: /Users/ioannis/research/projects/LSLGA/science/chaos/NGC5194-SDSS.fits
Done: /Users/ioannis/research/projects/LSLGA/science/chaos/NGC5457-SDSS.jpg
Done: /Users/ioannis/research/projects/LSLGA/science/chaos/NGC5457-SDSS.fits
Done: /Users/ioannis/research/projects/LSLGA/science/chaos/NGC3184-SDSS.jpg
Done: /Users/ioannis/research/projects/LSLGA/science/chaos/NGC3184-SDSS.fits


#### Read the data, do ellipse-fitting, and write out the results.

In [11]:
def ellipse(xcen, ycen, semia, semib, phi, x, y):
    xp = (x-xcen) * np.cos(phi) + (y-ycen) * np.sin(phi)
    yp = -(x-xcen) * np.sin(phi) + (y-ycen) * np.cos(phi)
    return (xp / semia)**2 + (yp/semib)**2 <= 1

In [36]:
def read_multiband(onegal, band=('g', 'r', 'i'), refband='r', 
                   pixscale=0.396, verbose=False):
    """Read the multiband data and pack into a dictionary.
    
    """
    from astropy.stats import sigma_clipped_stats
    from scipy.ndimage.morphology import binary_dilation

    galaxy = onegal['GALAXY']
    
    imgfile = os.path.join(datadir, '{}-SDSS.fits'.format(galaxy))
    if verbose:
        print('Reading {}'.format(imgfile))
    img = fitsio.read(imgfile)
    sumimg = np.sum(img, axis=0)
    
    _, H, W = img.shape
    
    if galaxy == 'NGC5194':    
        #xcen, ycen = 1050, 925
        xcen, ycen = 1012, 995
        semia = 444 * 1.5
        semib = semia * (1 - 0.3)
        phi = np.radians(35 + 90)
    else:
        xcen, ycen = W // 2, H // 2
        #semia = 2 * onegal['W1RSEMI'] / pixscale
        semia = 0.85 * onegal['D25'] * 60 / 2 / pixscale
        semib = semia * onegal['BA'] 
        if np.isfinite(onegal['PA']):
            phi = np.radians(onegal['PA'])
            #phi = np.radians(onegal['W1PA'])
        else:
            phi = 0.0

    # Mask the main galaxy
    ymask, xmask = np.ogrid[0:H, 0:W] # mask the galaxy
    galmask = ellipse(xcen, ycen, semia, semib, phi, xmask, ymask)
    
    if galaxy == 'NGC5194': # also mask NGC5195
        xcen2, ycen2, rad2 = 850, 1650, 300
        ymask2, xmask2 = np.ogrid[-ycen2:H-ycen2, -xcen2:W-xcen2]
        mask2 = (xmask2**2 + ymask2**2) <= rad2**2
        mn, med, sig = sigma_clipped_stats(ma.masked_array(
            sumimg, np.logical_or(galmask, mask2)), sigma=2.5)
        mask = np.logical_or( np.logical_and(np.abs((sumimg - med)) > 2.5 * sig, ~galmask),
                              mask2 )
    else:
        mn, med, sig = sigma_clipped_stats(ma.masked_array(
            sumimg, galmask), sigma=2.5)
        mask = np.logical_and(np.abs((sumimg - med)) > 2.5 * sig, ~galmask)
        
    # Now populate the output dictionary
    data = dict()
    data['band'] = band
    data['refband'] = refband
    data['pixscale'] = pixscale
    
    for filt, indx in zip( data['band'], (0, 1, 2) ):
        # Correct for dust and convert to surface brightness--
        print(onegal['GALAXY'], filt, 10**(0.4 * onegal['MWDUST_A{}'.format(filt.upper())]))
        sbimg = img[indx, :, :] * 10**(0.4 * onegal['MWDUST_A{}'.format(filt.upper())]) / data['pixscale']**2 # [nanomaggies/arcsec2]
        data[filt] = sbimg
        data['{}_mask'.format(filt)] = mask * 1
        data['{}_masked'.format(filt)] = ma.masked_array(sbimg * ~mask, mask)
        ma.set_fill_value(data['{}_masked'.format(filt)], 0)
        
    return data    

In [37]:
dd = read_multiband(sample[1])
#fitsio.write('junk.fits', dd['r_masked'], clobber=True)
#fitsio.write('junk2.fits', dd['r'], clobber=True)
#from LSLGA.mge import find_galaxy
#find_galaxy(dd['r_masked'], plot=True, quiet=False)

NGC5194 g 1.11275614173052
NGC5194 r 1.076465213629835
NGC5194 i 1.0568175092136585


In [14]:
def _build_sbprofiles_one(args):
    """Wrapper function for the multiprocessing."""
    return build_sbprofiles_one(*args)

In [19]:
def build_sbprofiles_one(onegal, band=('g', 'r', 'i'), refband='r', 
                         pixscale=0.396, clobber=False, verbose=False):
    from LSLGA.ellipse import ellipsefit_multiband

    galaxy = onegal['GALAXY']
    print('Working on {}'.format(galaxy))
        
    #maxsma = np.round(onegal['CUTOUT_SIZE'] / 2).astype(int)
    maxsma = np.round(0.85 * onegal['D25'] * 60 / 2 / pixscale / 2).astype(int) # [pixels]

    data = read_multiband(onegal, band=band, refband=refband,
                          pixscale=pixscale, verbose=verbose)
                
    # Read the ellipse-fitting results if they exist, otherwise do the fitting.
    ellipsefitfile = os.path.join(datadir, '{}-ellipsefit-fixed.p'.format(galaxy))
    if not os.path.exists(ellipsefitfile) or clobber:
        ell = ellipsefit_multiband(galaxy, datadir, data, onegal, maxsma=maxsma, 
                                   noellipsefit=True, verbose=verbose)
        LSLGA.io.write_ellipsefit(galaxy, datadir, ell, noellipsefit=True, 
                                  verbose=verbose)
    else:
        ell = LSLGA.io.read_ellipsefit(galaxy, datadir, noellipsefit=True, 
                                       verbose=verbose)
    return ell

In [16]:
def build_sbprofiles(sample, band=('g', 'r', 'i'), refband='r', 
                     pixscale=0.396, clobber=False, verbose=False):
    """Do ellipse-fitting and write out the surface brightness profiles.

    """
    from LSLGA.ellipse import ellipsefit_multiband

    sbargs = list()
    for onegal in sample:
        sbargs.append( (onegal, band, refband, pixscale, clobber, verbose) )

    if nproc > 1:
        p = multiprocessing.Pool(nproc)
        result = p.map(_build_sbprofiles_one, sbargs)
        p.close()
    else:
        result = list()
        for args in sbargs:
            result.append(_build_sbprofiles_one(sbargs))

In [20]:
%time build_sbprofiles(sample, clobber=False, verbose=True)

Working on NGC0628
Working on NGC5194
Working on NGC5457
Working on NGC3184
Reading /Users/ioannis/research/projects/LSLGA/science/chaos/NGC3184-SDSS.fits
Reading /Users/ioannis/research/projects/LSLGA/science/chaos/NGC0628-SDSS.fits
Reading /Users/ioannis/research/projects/LSLGA/science/chaos/NGC5457-SDSS.fits
Reading /Users/ioannis/research/projects/LSLGA/science/chaos/NGC5194-SDSS.fits
Finding the galaxy in the reference r-band image.
 Pixels used: 213939
 Peak Img[j, k]: 749 751
 Mean (j, k): 768.28 738.93
 Theta (deg): 97.2
 Astro PA (deg): 172.8
 Eps: 0.100
 Major axis (pix): 324.1
Finding the galaxy in the reference r-band image.
Finding the galaxy in the reference r-band image.
Finding the galaxy in the reference r-band image.
 Pixels used: 311829
 Peak Img[j, k]: 1000 1000
 Mean (j, k): 1017.18 989.85
 Theta (deg): 52.4
 Astro PA (deg): 37.6
 Eps: 0.331
 Major axis (pix): 423.0
 Pixels used: 366203
 Peak Img[j, k]: 1001 1001
 Mean (j, k): 996.89 995.07
 Theta (deg): 103.1
 Ast

In [26]:
def qa(sample, band=('g', 'r', 'i'), verbose=False):
    """Generate QA.
    
    """
    for onegal in sample:
        galaxy = onegal['GALAXY']
        data = read_multiband(onegal, band=band, verbose=verbose)
        ellipsefit = LSLGA.io.read_ellipsefit(galaxy, datadir, noellipsefit=True)
        
        png = os.path.join(datadir, '{}-ellipse-multiband.png'.format(galaxy))
        LSLGA.qa.display_multiband(data, ellipsefit=ellipsefit, png=png)
        
        smascale = onegal['DISTANCE'] * 1e3 / 206265
        png = os.path.join(datadir, '{}-ellipse-sbprofile.png'.format(galaxy))
        LSLGA.qa.display_ellipse_sbprofile(ellipsefit, png=png, smascale=smascale)

In [27]:
qa(sample, band=band)

Writing /Users/ioannis/research/projects/LSLGA/science/chaos/NGC0628-ellipse-multiband.png
Writing /Users/ioannis/research/projects/LSLGA/science/chaos/NGC0628-ellipse-sbprofile.png
Writing /Users/ioannis/research/projects/LSLGA/science/chaos/NGC5194-ellipse-multiband.png
Writing /Users/ioannis/research/projects/LSLGA/science/chaos/NGC5194-ellipse-sbprofile.png
Writing /Users/ioannis/research/projects/LSLGA/science/chaos/NGC5457-ellipse-multiband.png
Writing /Users/ioannis/research/projects/LSLGA/science/chaos/NGC5457-ellipse-sbprofile.png
Writing /Users/ioannis/research/projects/LSLGA/science/chaos/NGC3184-ellipse-multiband.png
Writing /Users/ioannis/research/projects/LSLGA/science/chaos/NGC3184-ellipse-sbprofile.png


In [28]:
galaxy = 'NGC3184'
ss = sample[3]
ellipsefit = LSLGA.io.read_ellipsefit(galaxy, datadir, noellipsefit=True)
sbprofile = LSLGA.qa.ellipse_sbprofile(ellipsefit)

In [62]:
solarmag = {'g': 5.15, 'r': 4.67, 'i': 4.56} # absolute solar magnitude
area = ellipsefit['i'].sarea * pixscale**2 # area of elliptical sector [arcsec2]
print(area)
10**(-0.4 * (sbprofile['mu_i'] + 2.5 * np.log10(area) - 5 * np.log10(ss['DISTANCE'] * 1e3) + 5 - solarmag['i']))

[ 0.313632    0.313632    0.313632    0.313632    0.313632    0.313632
  0.313632    0.313632    0.313632    0.313632    0.313632    0.313632
  0.313632    0.313632    0.313632    0.313632    0.313632    0.313632
  0.313632    0.313632    0.313632    0.313632    0.313632    0.313632
  0.313632    0.313632    0.313632    0.313632    0.313632    0.313632
  0.313632    0.313632    0.313632    0.313632    0.313632    0.56076643
  0.67852738  0.82101813  0.99343194  1.20205265  1.4544837   1.75992528
  2.12950959  2.5767066   3.11781499  3.77255613  4.56479292  5.52339944
  6.68331332  8.08680911  9.78503903 11.83989722 14.3451849  15.52277928
 15.4733774  15.57636719 15.7129756  15.77838114 15.9069802  15.84247461
 17.17413485 20.67663628 25.01873016 30.27266318 36.62992244]


array([1.85666188e+01, 1.85821132e+01, 1.85990756e+01, 1.86176359e+01,
       1.86379331e+01, 1.86601159e+01, 1.86843426e+01, 1.87107809e+01,
       1.87346087e+01, 1.74276930e+01, 1.74283385e+01, 1.74208162e+01,
       1.73836486e+01, 1.73338871e+01, 1.72781922e+01, 1.73111229e+01,
       1.70400299e+01, 1.69450117e+01, 1.65690027e+01, 1.63272362e+01,
       1.57286113e+01, 1.50714928e+01, 1.43304778e+01, 1.33665656e+01,
       1.24604254e+01, 1.15620459e+01, 1.07060118e+01, 9.82673439e+00,
       9.23189522e+00, 8.48715130e+00, 7.78287755e+00, 7.12335477e+00,
       6.48658586e+00, 5.90709483e+00, 5.39588654e+00, 2.80374951e+00,
       2.14566281e+00, 1.65696535e+00, 1.26277869e+00, 9.59595123e-01,
       7.41515279e-01, 5.65376714e-01, 4.38103686e-01, 3.40036242e-01,
       2.61546139e-01, 2.01097407e-01, 1.55088553e-01, 1.20284291e-01,
       9.30895221e-02, 7.31188273e-02, 5.71076621e-02, 4.41241293e-02,
       3.38212941e-02, 2.82131969e-02, 2.44911491e-02, 2.25587820e-02,
      

In [42]:
4 * np.pi * (ss['DISTANCE'] * 3.086e22)**2 * 10**(0.4 * (22.5 - sbprofile['mu_i']))

array([1.04509289e+50, 1.04596506e+50, 1.04691985e+50, 1.04796459e+50,
       1.04910710e+50, 1.05035574e+50, 1.05171943e+50, 1.05320761e+50,
       1.05454885e+50, 9.80984119e+49, 9.81020449e+49, 9.80597028e+49,
       9.78504912e+49, 9.75703897e+49, 9.72568897e+49, 9.74422523e+49,
       9.59163025e+49, 9.53814559e+49, 9.32649461e+49, 9.19040714e+49,
       8.85344829e+49, 8.48356411e+49, 8.06645554e+49, 7.52388086e+49,
       7.01382530e+49, 6.50813814e+49, 6.02628674e+49, 5.53135194e+49,
       5.19652405e+49, 4.77731656e+49, 4.38088924e+49, 4.00965170e+49,
       3.65122206e+49, 3.32503344e+49, 3.03728038e+49, 2.82177854e+49,
       2.61294675e+49, 2.44156271e+49, 2.25147469e+49, 2.07020440e+49,
       1.93566711e+49, 1.78580426e+49, 1.67439614e+49, 1.57250446e+49,
       1.46352573e+49, 1.36158227e+49, 1.27058151e+49, 1.19238656e+49,
       1.11659153e+49, 1.06122658e+49, 1.00290235e+49, 9.37617681e+48,
       8.70759214e+48, 7.86001707e+48, 6.80136352e+48, 6.30642931e+48,
      

In [ ]:

ai, bi = 0.006, 1.114 # r-i vs i coefficients
ml = 10**(ai + bi * sbprofile['ri']) # M/L
print(ml)